In [48]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


from public_data.models import Commune, OcsgeDiff, CommuneDiff, Ocsge, Departement

from django.contrib.gis.db.models.functions import Intersection, Area, Transform
from django.db.models import Sum, Q

In [45]:
city = Commune.objects.get(name="Arcachon")
city.id

12315

In [46]:
qs1 = OcsgeDiff.objects.filter(mpoly__intersects=city.mpoly)
qs1.count()

156

In [36]:
qs = OcsgeDiff.objects.filter(mpoly__intersects=city.mpoly)
qs = qs.annotate(intersection=Intersection("mpoly", city.mpoly))
qs = qs.annotate(intersection_area=Area(Transform("intersection", 2154)))
qs = qs.values("year_old", "year_new")
qs

<QuerySet [{'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, {'year_old': 2016, 'year_new': 2019}, '...(remaining elements truncated)...']>

In [44]:
qs = OcsgeDiff.objects.filter(mpoly__intersects=city.mpoly)
qs = qs.annotate(intersection=Intersection("mpoly", city.mpoly))
qs = qs.annotate(intersection_area=Area(Transform("intersection", 2154)))
qs = qs.values("year_old", "year_new")
qs = qs.annotate(new_artif=Sum("intersection_area", filter=Q(is_new_artif=True)))
qs = qs.annotate(new_natural=Sum("intersection_area", filter=Q(is_new_natural=True)))
qs

<QuerySet [{'year_old': 2016, 'year_new': 2019, 'new_artif': Area(sq_m=258436.29981787197), 'new_natural': Area(sq_m=16151.660983201931)}]>

In [39]:
print(qs[0]["new_natural"].sq_m)
print(qs[0]["new_artif"].sq_m)

5037.653377501541
13261.478738368618


In [41]:
qs = OcsgeDiff.objects.filter(mpoly__intersects=city.mpoly)
qs = qs.annotate(intersection=Intersection("mpoly", city.mpoly))
qs = qs.annotate(intersection_area=Area(Transform("intersection", 2154)))
qs = qs.values("year_old", "year_new")
qs = qs.annotate(
    new_artif=Sum("intersection_area", filter=Q(is_new_artif=True))
)
qs = qs.annotate(
    new_natural=Sum("intersection_area", filter=Q(is_new_natural=True))
)
qs

<QuerySet [{'year_old': 2016, 'year_new': 2019, 'new_artif': Area(sq_m=13261.478738368618), 'new_natural': Area(sq_m=5037.653377501541)}]>

In [32]:
qs = Ocsge.objects.filter(mpoly__intersects=city.mpoly)
qs.filter(year=2019, is_artificial=True)
qs.count()

18610

In [34]:
qs.first().id


404704

In [47]:
arcachon = Commune.objects.get(name="Arcachon")
OcsgeDiff.objects.filter(mpoly__intersects=city.mpoly, is_new_natural=True).count()

0

In [56]:
OcsgeDiff.objects.filter(
    mpoly__intersects=Departement.objects.get(name="Gironde").mpoly, is_new_natural=True
).count()

0

In [55]:
OcsgeDiff.objects.filter(mpoly__intersects=Departement.objects.get(name="Gironde").mpoly).count()

11886